Using FinBERT to scan through data article

In [ ]:
!pip install -q transformers
!pip install -q newspaper3k pandas numpy
!pip install -q lxml-html-clean

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 49.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 15.5 MB/s eta 0:00:00


In [ ]:
!pip install -q spacy

In [ ]:
import pandas as pd
import numpy as np
import torch
from newspaper import Article
import os

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
!ls

AAPL_main.csv  AAPL_relation.csv


In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [ ]:
from transformers import pipeline

pipe = pipeline("sentiment-analysis", model="ProsusAI/finbert", device=0 if torch.cuda.is_available() else -1)

In [ ]:
test_data = ["Adherents of the Dogs strategy assume that the highest yielding Dow stocks are unfairly and temporarily depressed. That presumably means they'll hold up better than the S&P 500. And that seems to be the case in 2022. Adding the Dogs stocks' 4% average dividend yield coming into 2022 plus their average 1.8% drop still left investors with a positive return of more than 2% on the stocks. Additionally, half of last year's Dogs, including International Business Machines (IBM), Chevron (CVX), Merck (MRK), Amgen (AMGN), and Coca-Cola (KO) posted gains in 2022. Chevron alone gained 53% on top of yielding 4.6%.",
             "The biggest dog of them all for 2023 is fallen communication services firm Verizon (VZ). The company, which has seen its stock drop 24% in 2022, is now yielding more than any other Dow stock: 6.5%. It's important to note, though, there's a reason Verizon's stock is shrinking. Verizon's profit is seen falling nearly 4% in 2022 and nearly 3% in 2023.",
             "the most average stock is stock ABC, which did not change at all"]

In [ ]:
result = pipe(test_data)
print(result)

[{'label': 'positive', 'score': 0.8819699287414551}, {'label': 'negative', 'score': 0.9748380184173584}, {'label': 'neutral', 'score': 0.9416248202323914}]


In [ ]:
!ls /content

In [ ]:
!rm AAPL_main.csv  AAPL_relation.csv

In [ ]:
import os
from google.colab import files

def upload_files_to_directory(target_dir):
    # Create the directory if it doesn't exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Upload files
    uploaded_files = files.upload()  # Opens file upload dialog
    for file_name, file_data in uploaded_files.items():
        file_path = os.path.join(target_dir, file_name)
        with open(file_path, 'wb') as f:
            f.write(file_data)
        print(f"Uploaded: {file_name} to {file_path}")

target_dir = "/content/old_data"

In [ ]:
upload_files_to_directory(target_dir)

Saving AAPL_main.csv to AAPL_main.csv
Saving AAPL_relation.csv to AAPL_relation.csv
Saving AMZN_main.csv to AMZN_main.csv
Saving AMZN_relation.csv to AMZN_relation.csv
Saving GOOGL_main.csv to GOOGL_main.csv
Saving GOOGL_relation.csv to GOOGL_relation.csv
Saving IBM_main.csv to IBM_main.csv
Saving IBM_relation.csv to IBM_relation.csv
Saving INTC_main.csv to INTC_main.csv
Saving INTC_relation.csv to INTC_relation.csv
Saving META_main.csv to META_main.csv
Saving META_relation.csv to META_relation.csv
Saving MSFT_main.csv to MSFT_main.csv
Saving MSFT_relation.csv to MSFT_relation.csv
Saving NFLX_main.csv to NFLX_main.csv
Saving NFLX_relation.csv to NFLX_relation.csv
Saving NVDA_main.csv to NVDA_main.csv
Saving NVDA_relation.csv to NVDA_relation.csv
Saving ORCL_main.csv to ORCL_main.csv
Saving ORCL_relation.csv to ORCL_relation.csv
Uploaded: AAPL_main.csv to /content/old_data/AAPL_main.csv
Uploaded: AAPL_relation.csv to /content/old_data/AAPL_relation.csv
Uploaded: AMZN_main.csv to /conten

In [ ]:
# from google.colab import files
# uploaded = files.upload()

Saving AAPL_main.csv to AAPL_main.csv
Saving AAPL_relation.csv to AAPL_relation.csv


In [ ]:
!ls /content/old_data

AAPL_main.csv	   GOOGL_main.csv      INTC_main.csv	  MSFT_main.csv      NVDA_main.csv
AAPL_relation.csv  GOOGL_relation.csv  INTC_relation.csv  MSFT_relation.csv  NVDA_relation.csv
AMZN_main.csv	   IBM_main.csv        META_main.csv	  NFLX_main.csv      ORCL_main.csv
AMZN_relation.csv  IBM_relation.csv    META_relation.csv  NFLX_relation.csv  ORCL_relation.csv


In [ ]:
def combine_dataframes_with_time_range(main_df, relation_df, start_time, end_time):
    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)

    main_df['publish_time'] = pd.to_datetime(main_df['publish_time'])
    relation_df['time'] = pd.to_datetime(relation_df['time'])
    filtered_rel = relation_df[relation_df['ticker'] == relation_df['source_ticker']]

    # Merge the two dataframes on `id` and `news_id`
    merged_df = main_df.merge(filtered_rel, left_on='id', right_on='news_id', how='inner')

    filtered_df = merged_df

    time_filtered_df = filtered_df[(filtered_df['publish_time'] >= start_time) & (filtered_df['publish_time'] <= end_time)]

    time_filtered_df = time_filtered_df.rename(columns={'ticker_x': 'ticker'})

    result_df = time_filtered_df[['id', 'publish_time', 'article_url', 'sentiment', 'ticker']]

    return result_df

In [ ]:
def analyze_sentiments_finbert_pipeline(merged_df, pipe, truncate_or_split="truncate"):
    # this one analyze with only the title, keywords and description
    result_data = []

    for i, row in enumerate(merged_df.itertuples(), start=1):
        if i % 1000 == 0:
            print(f"Processing article {i}/{len(merged_df)}")

        text = f"{row.title} {row.description} {row.keywords}"

        # print(text)

        if text != "Error" and isinstance(text, str):
            if truncate_or_split == "split":
                tokens = pipe.tokenizer(text, truncation=False, add_special_tokens=True)["input_ids"]

                if len(tokens) > 512:
                    chunk_size = 512
                    chunks = [text[start:start + chunk_size] for start in range(0, len(text), chunk_size)]

                    # use average score for labels instead
                    # Get predictions for each chunk
                    chunk_predictions = []
                    for chunk in chunks:
                        result = pipe(chunk)  # Send each chunk through the pipeline
                        if result[0]['label'] == "positive":
                            chunk_predictions.append(result[0]['score'])
                        elif result[0]['label'] == "negative":
                            chunk_predictions.append(- result[0]['score'])
                        else:
                            chunk_predictions.append(0)
                    # Use the most frequent label
                    predicted_score = sum(chunk_predictions) / len(chunk_predictions)
                    if predicted_score > 0.1:
                        predicted_sentiment = "positive"
                    elif predicted_score < 0.1:
                        predicted_sentiment = "negative"
                    else:
                        predicted_sentiment = "neutral"

                    # chunk_predictions = []
                    # for chunk in chunks:
                    #     result = pipe(chunk)  # Send each chunk through the pipeline
                    #     chunk_predictions.append(result[0]['label'])

                    # # Use the most frequent label
                    # predicted_sentiment = max(set(chunk_predictions), key=chunk_predictions.count)
                else:

                    result = pipe(text)
                    predicted_sentiment = result[0]['label']
                    # predicted_score = result[0]['score']
                    if predicted_sentiment == "positive":
                        predicted_score = result[0]['score']
                    elif predicted_sentiment == "negative":
                        predicted_score = -result[0]['score']
                    else:
                        predicted_score = 0
            else:
                result = pipe(text, truncation=True)
                predicted_sentiment = result[0]['label']
                # predicted_score = result[0]['score']
                if predicted_sentiment == "positive":
                    predicted_score = result[0]['score']
                elif predicted_sentiment == "negative":
                    predicted_score = -result[0]['score']
                else:
                    predicted_score = 0

            result_data.append({
                "ticker": row.ticker,
                "sentiment": predicted_sentiment,
                "sentiment_score": predicted_score,
                "news_id": row.id,
                "time": row.publish_time,
                "title": row.title,
                'article_url': row.article_url
            })

    result_df = pd.DataFrame(result_data, columns=['ticker', 'sentiment', 'sentiment_score', 'news_id', 'time', "title", 'article_url'])

    return result_df


In [ ]:
os.rename('/content/old_data/AAPL_main (1).csv', '/content/old_data/AAPL_main.csv')
os.rename('/content/old_data/AAPL_relation (1).csv', '/content/old_data/AAPL_relation.csv')

In [ ]:
new_directory = "/content/title_sentiments"

In [ ]:
if not os.path.exists(new_directory):
        os.makedirs(new_directory)

In [ ]:
!ls

AAPL_main.csv	   GOOGL_relation.csv  META_main.csv	  NFLX_relation.csv  ORCL_relation.csv
AAPL_relation.csv  IBM_main.csv        META_relation.csv  NVDA_main.csv      title_sentiments
AMZN_main.csv	   IBM_relation.csv    MSFT_main.csv	  NVDA_relation.csv
AMZN_relation.csv  INTC_main.csv       MSFT_relation.csv  old_data
GOOGL_main.csv	   INTC_relation.csv   NFLX_main.csv	  ORCL_main.csv


In [ ]:

for file_name in os.listdir(target_dir):
    file_path = os.path.join(target_dir, file_name)
    if os.path.isfile(file_path):  # Check if it is a file
        if "main" in file_name:
            # print(file_name)
            ticker = file_name.split("_")[0]
            print(ticker)
            df_news = pd.read_csv(file_path)

            result_df = analyze_sentiments_finbert_pipeline(df_news, pipe, truncate_or_split="split")
            result_df.to_csv(f"{new_directory}/{ticker}_title_sentiments.csv", index=False)



AMZN
Processing article 1000/10885
Processing article 2000/10885
Processing article 3000/10885
Processing article 4000/10885
Processing article 5000/10885
Processing article 6000/10885
Processing article 7000/10885
Processing article 8000/10885
Processing article 9000/10885
Processing article 10000/10885
MSFT
Processing article 1000/8914
Processing article 2000/8914
Processing article 3000/8914
Processing article 4000/8914
Processing article 5000/8914
Processing article 6000/8914
Processing article 7000/8914
Processing article 8000/8914
IBM
Processing article 1000/1991
META
Processing article 1000/6957
Processing article 2000/6957
Processing article 3000/6957
Processing article 4000/6957
Processing article 5000/6957
Processing article 6000/6957
AAPL
Processing article 1000/9881
Processing article 2000/9881
Processing article 3000/9881
Processing article 4000/9881
Processing article 5000/9881
Processing article 6000/9881
Processing article 7000/9881
Processing article 8000/9881
Processi

In [ ]:
for file_name in os.listdir(new_directory):
    file_path = os.path.join(new_directory, file_name)
    if os.path.isfile(file_path):
        files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls /content/title_sentiments

AAPL_title_sentiments.csv   INTC_title_sentiments.csv  NVDA_title_sentiments.csv
AMZN_title_sentiments.csv   META_title_sentiments.csv  ORCL_title_sentiments.csv
GOOGL_title_sentiments.csv  MSFT_title_sentiments.csv
IBM_title_sentiments.csv    NFLX_title_sentiments.csv


In [ ]:
start_time = "2023-07-01"
end_time = "2023-08-01"

In [ ]:
df_news = pd.read_csv('/content/old_data/AAPL_main.csv')
time_filtered_df = df_news[(df_news['publish_time'] >= start_time) & (df_news['publish_time'] <= end_time)]

In [ ]:
print(time_filtered_df)

                                               id         publish_time  \
5266  fDsmdg7YPIjgOHIrX87Ywwjcn1MVHbmUxkkCNKfC2uU  2023-07-31 22:10:00   
5267  -1wCRYWVX8nwx7jdWzqGpsYm1HJ_sW8BVXe8ToFWuDM  2023-07-31 21:12:00   
5268  70MKVLhoxKbcYZT3gJImUVdlPzssXRfujYBGxI18yQQ  2023-07-31 20:57:00   
5269  evlHmM8758OCYWqaDEGUJBScOoBiS46KqYcLtgnzgvs  2023-07-31 20:31:00   
5270  88S57GotBWs7gAtLSxarV4laYWbgkrbb00-axBBUJwQ  2023-07-31 19:22:40   
...                                           ...                  ...   
5674  ptM15lww-Lp7SBt3e2MB28_Zc3fIB7LqoCoS5JODxbs  2023-07-01 10:45:00   
5675  _OHmUn9VPu1STFeWtEVzxLnaAuHIcPrtf3cbEK4EM5c  2023-07-01 10:45:00   
5676  3RwOP0zjyWvY9fOVYfa5aOUAwWtzKyyamipSDl9DeRE  2023-07-01 05:07:46   
5677  KABug156Fwg9Ln7yL-mbvI7vDeFrv0IF2HwFBxBzrOk  2023-07-01 04:29:12   
5678  bpPo13eIbhVWJtemgoV76U8yMjjULURLhI9V9q5-b70  2023-07-01 04:25:00   

                                                  title  \
5266  San Francisco's push to turn office buildings 

In [ ]:
result_df = analyze_sentiments_finbert_pipeline(time_filtered_df, pipe, truncate_or_split="split")

Processing article 100/413
Processing article 200/413
Processing article 300/413
Processing article 400/413


In [ ]:
print(result_df)

    ticker sentiment  sentiment_score  \
0     AAPL   neutral         0.776678   
1     AAPL   neutral         0.899154   
2     AAPL  positive         0.944890   
3     AAPL  positive         0.844155   
4     AAPL  positive         0.508103   
..     ...       ...              ...   
408   AAPL   neutral         0.900608   
409   AAPL   neutral         0.875981   
410   AAPL   neutral         0.552383   
411   AAPL   neutral         0.869364   
412   AAPL  negative         0.910636   

                                         news_id                 time  \
0    fDsmdg7YPIjgOHIrX87Ywwjcn1MVHbmUxkkCNKfC2uU  2023-07-31 22:10:00   
1    -1wCRYWVX8nwx7jdWzqGpsYm1HJ_sW8BVXe8ToFWuDM  2023-07-31 21:12:00   
2    70MKVLhoxKbcYZT3gJImUVdlPzssXRfujYBGxI18yQQ  2023-07-31 20:57:00   
3    evlHmM8758OCYWqaDEGUJBScOoBiS46KqYcLtgnzgvs  2023-07-31 20:31:00   
4    88S57GotBWs7gAtLSxarV4laYWbgkrbb00-axBBUJwQ  2023-07-31 19:22:40   
..                                           ...                  ...

In [ ]:
print(result_df)

    ticker sentiment                                      news_id  \
0     AAPL   neutral  fDsmdg7YPIjgOHIrX87Ywwjcn1MVHbmUxkkCNKfC2uU   
1     AAPL   neutral  -1wCRYWVX8nwx7jdWzqGpsYm1HJ_sW8BVXe8ToFWuDM   
2     AAPL  positive  70MKVLhoxKbcYZT3gJImUVdlPzssXRfujYBGxI18yQQ   
3     AAPL  positive  evlHmM8758OCYWqaDEGUJBScOoBiS46KqYcLtgnzgvs   
4     AAPL  positive  88S57GotBWs7gAtLSxarV4laYWbgkrbb00-axBBUJwQ   
..     ...       ...                                          ...   
408   AAPL   neutral  ptM15lww-Lp7SBt3e2MB28_Zc3fIB7LqoCoS5JODxbs   
409   AAPL   neutral  _OHmUn9VPu1STFeWtEVzxLnaAuHIcPrtf3cbEK4EM5c   
410   AAPL   neutral  3RwOP0zjyWvY9fOVYfa5aOUAwWtzKyyamipSDl9DeRE   
411   AAPL   neutral  KABug156Fwg9Ln7yL-mbvI7vDeFrv0IF2HwFBxBzrOk   
412   AAPL  negative  bpPo13eIbhVWJtemgoV76U8yMjjULURLhI9V9q5-b70   

                    time                                              title  \
0    2023-07-31 22:10:00  San Francisco's push to turn office buildings ...   
1    2023-07-

In [ ]:
result_df = analyze_sentiments_finbert_pipeline(df_news, pipe, truncate_or_split="split")

Processing article 100/9881
Processing article 200/9881
Processing article 300/9881
Processing article 400/9881
Processing article 500/9881
Processing article 600/9881
Processing article 700/9881
Processing article 800/9881
Processing article 900/9881
Processing article 1000/9881
Processing article 1100/9881
Processing article 1200/9881
Processing article 1300/9881
Processing article 1400/9881
Processing article 1500/9881
Processing article 1600/9881
Processing article 1700/9881
Processing article 1800/9881
Processing article 1900/9881
Processing article 2000/9881
Processing article 2100/9881
Processing article 2200/9881
Processing article 2300/9881
Processing article 2400/9881
Processing article 2500/9881
Processing article 2600/9881
Processing article 2700/9881
Processing article 2800/9881
Processing article 2900/9881
Processing article 3000/9881
Processing article 3100/9881
Processing article 3200/9881
Processing article 3300/9881
Processing article 3400/9881
Processing article 3500

In [ ]:
print(result_df)

     ticker sentiment                                            news_id  \
0      AAPL   neutral  0d2a16f0328775bebe96ecbd57fb1ee7b98fae09d2ee73...   
1      AAPL  negative  55d38db6df60d647da21ac34f9942bfc7729d3196b10ff...   
2      AAPL   neutral  4ef47f7a1d56daecdf73f02eb2ceb206df1e629e6f04ba...   
3      AAPL   neutral  1358145bef572d9b1580795b261a011c39169815dae386...   
4      AAPL  positive  ebed3bc762a748dd4eba4d38230efe87101b1c28ad4096...   
...     ...       ...                                                ...   
9876   AAPL   neutral        ciYU0qzpWQXA34UNHWKbBqQZyH5RZ-bf4mMIchoFOkI   
9877   AAPL  negative        SNe7uhcmDkVap9wLTLCO6DSGNj9JzaUu13tNIyocUv4   
9878   AAPL  positive        CD9U-8i7O7pgOuCfOJ5CmyHfuyaxJTvuE8sgKliJmH8   
9879   AAPL  negative        I3UTSe9ArLGw1g0gXXp_tLt_8463iT11N9HHQK3u95c   
9880   AAPL   neutral        HOUyxErrGAILFLEPNmGTNsTBrUr-rAWd8Q4rbRHF9Pw   

                     time                                              title  \
0     2

In [ ]:
test_df = df_news[(df_news['publish_time'] >= "2023-11-01") & (df_news['publish_time'] <= "2023-11-02")]

In [ ]:
print(test_df)

                                               id         publish_time  \
3909  0-aHwVSJBNGZk5_P03r59htX5WNMfRbEgjpIpSle7B4  2023-11-01 17:59:19   
3910  oMXj0WrB2_7D4QpIjyWoqPUTpNJh9Tnl6vnIB0_4c08  2023-11-01 17:01:26   
3911  FX8kvnBIoo9honbMqdmhM364PCoekfnXsjBnvDwGWyA  2023-11-01 16:35:00   
3912  Rv_EAQvX6uJi5K3wt_iNxF2B_OH0u-Bo6jZMta4pWm0  2023-11-01 15:40:00   
3913  R33sodoWNHCz7nTJGcrlV9UZdS0x6En7t7dieKp2yTs  2023-11-01 13:54:00   
3914  v7Y4pXj4BZ6UvCDkGAz_Q6Ude63fcSzPnoNtORlw2xw  2023-11-01 13:20:00   
3915  qrZ1aBMve1-TTHxCbnqVYFE5Ow4p0Q1mnFF_6X_MphI  2023-11-01 12:49:00   
3916  yzUfPc-DfSg5ZerRhsVFQMU2zMKKEr72y--K4P1ktpk  2023-11-01 12:18:00   
3917  JX4-Y5xGsw5fsEUHfoveCTCGtVpdIWOma6EXpzWEDyM  2023-11-01 12:12:59   
3918  0g8QtgYbIJXTlJXQ3ZOxXwMTFl7TIwX7Nhg_qLpC6U0  2023-11-01 11:15:00   
3919  QAlTP1R8Gu0AyrspSx8tlLe3pRC-YmWeSjOnypDv_Js  2023-11-01 10:40:00   
3920  2Ig5WSaYHdiNKlVgZuYqA1CVlk7hTLBTX_qDEki5kiU  2023-11-01 10:33:00   
3921  IaFSqd3YwMW6xX_mTUSKn3WNBs_QqKnk

In [ ]:
tdf_r = analyze_sentiments_finbert_pipeline(test_df, pipe, truncate_or_split="split")

Apple Earnings: Why Guidance Will Be Key Investors are hoping the iPhone maker can return to growth during the important holiday quarter. investing
Apple Q4 Earnings Preview: All Eyes On iPhone 15 Technology giant Apple Inc (NASDAQ:AAPL) is set to report fourth-quarter financial results after the market close Thursday.
Here's a look at the earning estimates from analysts, what analysts are saying ahead of the report and key items for investors to watch.
Apple Earnings Estimates: Analysts expect Apple to report fourth-quarter revenue of $89.27 billion, according to data from Benzinga Pro.
Apple reported fourth-quarter revenue of $90.15 billion in the previous fiscal year. The company has beaten revenue estimates from analysts in six of the last seven quarters.
Analysts expect Apple to report fourth-quarter earnings per share of $1.39, compared to $1.29 reported in the comparable quarter last year. The company has beaten earnings per share estimates in six of the last seven quarters.
Rel

In [ ]:
print(tdf_r)

   ticker sentiment                                      news_id  \
0    AAPL  positive  0-aHwVSJBNGZk5_P03r59htX5WNMfRbEgjpIpSle7B4   
1    AAPL  positive  oMXj0WrB2_7D4QpIjyWoqPUTpNJh9Tnl6vnIB0_4c08   
2    AAPL  negative  FX8kvnBIoo9honbMqdmhM364PCoekfnXsjBnvDwGWyA   
3    AAPL  negative  Rv_EAQvX6uJi5K3wt_iNxF2B_OH0u-Bo6jZMta4pWm0   
4    AAPL   neutral  R33sodoWNHCz7nTJGcrlV9UZdS0x6En7t7dieKp2yTs   
5    AAPL  negative  v7Y4pXj4BZ6UvCDkGAz_Q6Ude63fcSzPnoNtORlw2xw   
6    AAPL   neutral  qrZ1aBMve1-TTHxCbnqVYFE5Ow4p0Q1mnFF_6X_MphI   
7    AAPL   neutral  yzUfPc-DfSg5ZerRhsVFQMU2zMKKEr72y--K4P1ktpk   
8    AAPL   neutral  JX4-Y5xGsw5fsEUHfoveCTCGtVpdIWOma6EXpzWEDyM   
9    AAPL   neutral  0g8QtgYbIJXTlJXQ3ZOxXwMTFl7TIwX7Nhg_qLpC6U0   
10   AAPL   neutral  QAlTP1R8Gu0AyrspSx8tlLe3pRC-YmWeSjOnypDv_Js   
11   AAPL   neutral  2Ig5WSaYHdiNKlVgZuYqA1CVlk7hTLBTX_qDEki5kiU   
12   AAPL   neutral  IaFSqd3YwMW6xX_mTUSKn3WNBs_QqKnkBjwlnggpPAM   
13   AAPL   neutral  uSPsRAm6Xd5AvvzRo5D5nVStAyA

In [32]:
!ls /content/total_sentiments

GOOGL_total_sentiments.csv  INTC_total_sentiments.csv  NFLX_total_sentiments.csv
IBM_total_sentiments.csv    META_total_sentiments.csv  ORCL_total_sentiments.csv
